# IRIS 데이터 *분류
독립변수: 꽃잎길이,넓이, 꽃받침 넓이, 길이

종속변수: 붓꽃 분류

## 범주형일때 손실함수
분류문제에선 'categorical_crossentropy' 사용

정확도 'accuracy'

keras에서 분류문제를 사용한다는 표시 

'categorical'_크로스엔트로피

loss = 'crossentropy'으로만 돌리면 오류남

# 손실함수 (loss function)
: 모델의 출력값(예측값)과 실제값의 오차.
따라서 손실함수를 줄이는 것 중요

```
- 범주형 교차 엔트로피
: Dense(n, activation="softmax"),loss="categorical_crossentropy"
   y 출력값이 2개 이상인 경우 사용
   out layer = "softmax" (0과 1사이의 값이 나오고, 그 합은 1)
   one-hot encoding class 결과 출력,
   one-hot encoding 필요
   각 클래스별 positive 확률값 출력

   ex) Dense=3 일때, 
   label [ 0,0,1 ]과 출력값 [ 0.2, 0.3, 0.5 ]

   실제값과 예측값이 모두 같은경우 손실함수는 0
   실제값과 예측값이 다른경우 양의 무한대로 발산 

```
```

- Sparse 범주형 교차 엔트로피
Dense(n, activation="softmax"),loss="sparse_categorical_crossentropy"
   y class값이 2개 이상인 경우 사용
   정수타입의 결과 출력
   one-hot encoding 필요하지 않음
   한 샘플에 여러 class 존재할 경우
   out layer = "softmax" (0과 1사이의 값이 나오고, 그 합은 1)
   각 클래스별 positive 확률값 출력

   ex) Dense=3 일때, 
   label [ 0,1,2 ]과 출력값 [ 0.2, 0.3, 0.5 ]
```
```

- 이항 교차 엔트로피
: Dense(1, activation="sigmoid"), loss="binary_crossentropy"
   label들이 독립적일때 사용
   out layer = "sigmoid"
   y값이 0,1인 이진분류 훈련
```


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import optimizers
from tensorflow.keras.layers import Dense, Input

## 데이터 준비

In [ ]:
!wget https://raw.githubusercontent.com/dhrim/MDC_2021/master/material/deep_learning/iris.csv

--2021-12-22 01:23:57--  https://raw.githubusercontent.com/dhrim/MDC_2021/master/material/deep_learning/iris.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2720 (2.7K) [text/plain]
Saving to: ‘iris.csv’

iris.csv            100%[===================>]   2.66K  --.-KB/s    in 0s      

2021-12-22 01:23:57 (36.2 MB/s) - ‘iris.csv’ saved [2720/2720]



In [ ]:
# 리눅스 언어
!ls -al
!head iris.csv

total 20
drwxr-xr-x 1 root root 4096 Dec 22 01:23 .
drwxr-xr-x 1 root root 4096 Dec 22 01:17 ..
drwxr-xr-x 4 root root 4096 Dec  3 14:33 .config
-rw-r--r-- 1 root root 2720 Dec 22 01:23 iris.csv
drwxr-xr-x 1 root root 4096 Dec  3 14:33 sample_data
septal_length,septal_width,petal_length,petal_width,setosa,versicolor,virginica
6.4,2.8,5.6,2.2,0,0,1
5.0,2.3,3.3,1.0,0,1,0
4.9,2.5,4.5,1.7,0,0,1
4.9,3.1,1.5,0.1,1,0,0
5.7,3.8,1.7,0.3,1,0,0
4.4,3.2,1.3,0.2,1,0,0
5.4,3.4,1.5,0.4,1,0,0
6.9,3.1,5.1,2.3,0,0,1
6.7,3.1,4.4,1.4,0,1,0


In [ ]:
iris = pd.read_csv("iris.csv")
iris.head()

,septal_length,septal_width,petal_length,petal_width,setosa,versicolor,virginica
0,6.4,2.8,5.6,2.2,0,0,1
1,5.0,2.3,3.3,1.0,0,1,0
2,4.9,2.5,4.5,1.7,0,0,1
3,4.9,3.1,1.5,0.1,1,0,0
4,5.7,3.8,1.7,0.3,1,0,0


종속변수가 3개인 데이터

one-hot encoding 처리된 형식

In [ ]:
iris.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120 entries, 0 to 119
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   septal_length  120 non-null    float64
 1   septal_width   120 non-null    float64
 2   petal_length   120 non-null    float64
 3   petal_width    120 non-null    float64
 4   setosa         120 non-null    int64  
 5   versicolor     120 non-null    int64  
 6   virginica      120 non-null    int64  
dtypes: float64(4), int64(3)
memory usage: 6.7 KB


In [ ]:
# 넘파이 배열로 받아옴
data = iris.to_numpy()
print(data.shape)
print(data[:5])

(120, 7)
[[6.4 2.8 5.6 2.2 0.  0.  1. ]
 [5.  2.3 3.3 1.  0.  1.  0. ]
 [4.9 2.5 4.5 1.7 0.  0.  1. ]
 [4.9 3.1 1.5 0.1 1.  0.  0. ]
 [5.7 3.8 1.7 0.3 1.  0.  0. ]]


### 데이터 나누기

In [ ]:
# x,y 나누기 
x = data[:,:4]
y = data[:,4:]

# train, test 데이터 나누기
split_index = 100

train_x, test_x = x[:split_index], x[split_index:]
train_y, test_y = y[:split_index], y[split_index:]

In [ ]:
print(train_x.shape)
print(train_y.shape)

(100, 4)
(100, 3)


## 모델 설정

In [ ]:
# 입력 4개 (독립변수 4개), 출력 3개 (종속변수 3개)
model = keras.Sequential()
model.add(Input(4))
model.add(Dense(10, activation='tanh'))
model.add(Dense(10, activation='tanh'))
model.add(Dense(3, activation='softmax')) # softmax: 출력값들 중 가장 큰 값 (0~1), 출력층에만 사용됨

In [ ]:
# 분류문제에선 'categorical_crossentropy' 사용
# 정확도 'accuracy'
# keras에서 분류문제를 사용한다는 표시 *categorical*_크로스엔트로피
# loss = 'crossentropy'으로만 돌리면 오류남
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(train_x, train_y, epochs=200, verbose=0)

loss, acc = model.evaluate(test_x, test_y)

print("loss :", loss)
print("acc :", acc)

1/1 [==============================] - 0s 184ms/step - loss: 0.1047 - accuracy: 1.0000
loss : 0.10471087694168091
acc : 1.0


In [ ]:
y_ = model.predict(test_x)
print(y_)
print(np.argmax(y_, axis=1)) # argmax 넘파이 최대값

[[9.6206725e-01 3.7705105e-02 2.2767970e-04]
 [7.9625189e-02 8.7921280e-01 4.1162018e-02]
 [9.2638284e-01 7.3129140e-02 4.8804976e-04]
 [5.2833773e-02 9.0044647e-01 4.6719756e-02]
 [9.6654832e-01 3.3244599e-02 2.0710487e-04]
 [9.6922505e-01 3.0596033e-02 1.7882881e-04]
 [9.7173595e-01 2.8099010e-02 1.6490593e-04]
 [9.7187144e-01 2.7961481e-02 1.6711226e-04]
 [9.7462721e-02 8.6527079e-01 3.7266530e-02]
 [9.6100533e-01 3.8753454e-02 2.4115863e-04]
 [8.5674496e-03 2.9725832e-01 6.9417429e-01]
 [4.8183125e-02 8.3245474e-01 1.1936212e-01]
 [9.5683360e-01 4.2902824e-02 2.6363449e-04]
 [7.1256008e-04 2.7228529e-02 9.7205895e-01]
 [9.6722591e-01 3.2563850e-02 2.1024213e-04]
 [2.5746500e-02 6.5026879e-01 3.2398468e-01]
 [5.7649888e-02 8.3653873e-01 1.0581134e-01]
 [9.6103936e-01 3.8714163e-02 2.4645249e-04]
 [9.6272105e-01 3.7055545e-02 2.2344645e-04]
 [5.4732706e-02 9.0689135e-01 3.8376015e-02]]
[0 1 0 1 0 0 0 0 1 0 2 1 0 2 0 1 1 0 0 1]


## category index
종속변수 3개를 카테고리 인덱스 하나로 묶어줌

종속변수 1개에 class 3개

one-hot encoding 처리되지 않은 형식

In [ ]:
!wget https://raw.githubusercontent.com/dhrim/MDC_2021/master/material/deep_learning/iris_with_category_index.csv

--2021-12-22 02:11:43--  https://raw.githubusercontent.com/dhrim/MDC_2021/master/material/deep_learning/iris_with_category_index.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2218 (2.2K) [text/plain]
Saving to: ‘iris_with_category_index.csv’

iris_with_category_ 100%[===================>]   2.17K  --.-KB/s    in 0s      

2021-12-22 02:11:43 (28.1 MB/s) - ‘iris_with_category_index.csv’ saved [2218/2218]



In [ ]:
!ls -al
!head iris_with_category_index.csv

total 24
drwxr-xr-x 1 root root 4096 Dec 22 02:11 .
drwxr-xr-x 1 root root 4096 Dec 22 01:17 ..
drwxr-xr-x 4 root root 4096 Dec  3 14:33 .config
-rw-r--r-- 1 root root 2720 Dec 22 01:23 iris.csv
-rw-r--r-- 1 root root 2218 Dec 22 02:11 iris_with_category_index.csv
drwxr-xr-x 1 root root 4096 Dec  3 14:33 sample_data
septal_length,septal_width,petal_length,petal_width,class
6.4,2.8,5.6,2.2,2
5.0,2.3,3.3,1.0,1
4.9,2.5,4.5,1.7,2
4.9,3.1,1.5,0.1,0
5.7,3.8,1.7,0.3,0
4.4,3.2,1.3,0.2,0
5.4,3.4,1.5,0.4,0
6.9,3.1,5.1,2.3,2
6.7,3.1,4.4,1.4,1


In [ ]:
iris = pd.read_csv("iris_with_category_index.csv")
iris.head()

,septal_length,septal_width,petal_length,petal_width,class
0,6.4,2.8,5.6,2.2,2
1,5.0,2.3,3.3,1.0,1
2,4.9,2.5,4.5,1.7,2
3,4.9,3.1,1.5,0.1,0
4,5.7,3.8,1.7,0.3,0


In [ ]:
data = iris.to_numpy()
print(data.shape)
print(data[:5])

(120, 5)
[[6.4 2.8 5.6 2.2 2. ]
 [5.  2.3 3.3 1.  1. ]
 [4.9 2.5 4.5 1.7 2. ]
 [4.9 3.1 1.5 0.1 0. ]
 [5.7 3.8 1.7 0.3 0. ]]


In [ ]:
x = data[:,:4]
y = data[:,4:]

split_index = 100

train_x, test_x = x[:split_index], x[split_index:]
train_y, test_y = y[:split_index], y[split_index:]

print(train_x.shape)
print(train_y.shape)
print(test_x.shape)
print(test_y.shape)

(100, 4)
(100, 1)
(20, 4)
(20, 1)


In [ ]:
# 종속변수로 1개만 들어가도 Dense(3)으로 설정
# 단, loss=*sparse*_카테고리컬_크로스엔트로피 (sparse로 표시)
model = keras.Sequential()
model.add(Input(4))
model.add(Dense(10, activation='tanh'))
model.add(Dense(10, activation='tanh'))
model.add(Dense(3, activation='softmax')) # 1이 아니고 클래스 수 3이다

# model.compile(optimizer="SGD", loss="categorical_crossentropy", metrics=["accuracy"])는 오류남
model.compile(optimizer="SGD", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
model.summary()

model.fit(train_x, train_y, epochs=1000, verbose=0, batch_size=20)

loss, acc = model.evaluate(test_x, test_y)
print("loss=", loss)
print("acc=", acc)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 10)                50        
                                                                 
 dense_4 (Dense)             (None, 10)                110       
                                                                 
 dense_5 (Dense)             (None, 3)                 33        
                                                                 
Total params: 193
Trainable params: 193
Non-trainable params: 0
_________________________________________________________________
1/1 [==============================] - 0s 125ms/step - loss: 0.0305 - accuracy: 1.0000
loss= 0.03051740862429142
acc= 1.0


In [ ]:
y_ = model.predict(test_x)
print(y_)
print(np.argmax(y_, axis=1))

[[9.93573248e-01 6.29249495e-03 1.34223679e-04]
 [1.34519488e-02 9.85262394e-01 1.28558627e-03]
 [9.82520401e-01 1.72649454e-02 2.14738102e-04]
 [7.88637623e-03 9.90965366e-01 1.14830001e-03]
 [9.94501233e-01 5.37061598e-03 1.28104730e-04]
 [9.95299578e-01 4.58093733e-03 1.19519573e-04]
 [9.95879769e-01 4.00558906e-03 1.14734088e-04]
 [9.95931685e-01 3.95348435e-03 1.14849434e-04]
 [2.00024974e-02 9.78859067e-01 1.13842334e-03]
 [9.93408084e-01 6.45586289e-03 1.36096758e-04]
 [3.90491274e-04 2.95990765e-01 7.03618765e-01]
 [4.43235924e-03 9.91803110e-01 3.76448105e-03]
 [9.92195964e-01 7.65969232e-03 1.44315592e-04]
 [9.14926932e-05 6.46564811e-02 9.35252011e-01]
 [9.94439483e-01 5.43045951e-03 1.30053086e-04]
 [2.05528806e-03 9.56210375e-01 4.17343304e-02]
 [8.18611495e-03 9.89900529e-01 1.91329629e-03]
 [9.93480682e-01 6.38195267e-03 1.37332710e-04]
 [9.93926346e-01 5.94263384e-03 1.31002220e-04]
 [6.99622789e-03 9.91762996e-01 1.24076998e-03]]
[0 1 0 1 0 0 0 0 1 0 2 1 0 2 0 1 1 0 0 

두 경우 모두 acc가 1이 나와, loss기준으로 평가한 결과 종속변수를 카테고리 인덱스 하나로 묶어준 sparse_categorical_crossentropy가 loss가 더 낮아 더 좋은결과를 얻은것으로 보인다